# Run Regressions

This short notebook runs ridge regressions on the pre-featurized data matrix using a 5-fold cross-validation approach for all 7 labels used in the study. It saves `.data` files for the cross-validation results (used by Fig 2) and the test set results. It also saves a CSV of test set results that are used for Table S2 and Figure 3.

## Settings

Here are the settings you can adjust when running this notebook:
- ``num_threads``: If running on a multi-core machine, change this from ``None`` to an ``int`` in order to set the max number of threads to use
- ``subset_[n,feat]``: If you want to subset the training set data for quick tests/debugging, specify that here using the `slice` object. `slice(None)` implies no subsetting of the ~80k observations for each label that are in the training set. `subset_n` slices observations; `subset_feat` subsets features.
- ``overwrite``: By default, this code will raise an error if the file you are saving already exists. If you would like to disable that and overwrite existing data files, change `overwrite` to `True`.
- ``fixed_lambda``: If True, only run the lambda that was previously chosen. Will throw an error if you haven't already generated a results file.
- ``labels_to_run``: By default, this notebook will loop through all the labels. If you would like, you can reduce this list to only loop through a subset of them, by changing ``"all"`` to a list of task names, e.g. ``["housing", "treecover"]``

In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
import os
import pickle
from pathlib import Path

from mosaiks import transforms
from mosaiks.utils import OVERWRITE_EXCEPTION
from mosaiks.utils.imports import *
from sklearn.metrics import r2_score
from threadpoolctl import threadpool_limits

In [7]:
num_threads = None

subset_n = slice(None)
subset_feat = slice(None)

overwrite = None
fixed_lambda = False

labels_to_run = "all"

if overwrite is None:
    overwrite = os.getenv("MOSAIKS_OVERWRITE", False)
if labels_to_run == "all":
    labels_to_run = c.app_order

if num_threads is not None:
    threadpool_limits(num_threads)
    os.environ["NUMBA_NUM_THREADS"] = str(num_threads)

solver = solve.ridge_regression

### Define Output Location
If you change this location, the template must contain the following sub-strings that get filled in within the loop below:
- ``{save_dir}`` The directory in which to save.
- ``{reg_type}``: Filled with ``scatter`` for CV and ``testset`` for test set.
- ``{label}``: Will be filled with each label you are predicting.
- ``{variable}``: The variable name for each label you are predicting.
- ``{sampling}``: The sampling scheme for each label you are predicting.
- ``{subset}``: Adjusted based on whether you are running on the full dataset or a subset.

In [4]:
save_patt = join(
    "{save_dir}",
    "outcomes_{{reg_type}}_obsAndPred_{label}_{variable}_CONTUS_16_640_{sampling}_"
    f"{c.sampling['n_samples']}_{c.sampling['seed']}_random_features_{c.features['random']['patch_size']}_"
    f"{c.features['random']['seed']}{{subset}}.data",
)

## Load Random Feature Data

This loads our feature matrix `X` for both POP and UAR samples

In [5]:
X = {}
latlons = {}

X["UAR"], latlons["UAR"] = io.get_X_latlon(c, "UAR")
X["POP"], latlons["POP"] = io.get_X_latlon(c, "POP")

## Run regressions

The following loop will:
1. Load the appropriate labels
2. Merge them with the feature matrix
3. Remove test set observations
4. Run a ridge regression on the training/validation set, sweeping over a range of possible regularization parameters using 5-fold Cross-Validation and clipping predictions to pre-specified bounds.
5. Save the out-of-sample predictions and the observations for use in Figure 2.

**Note**: We drop observations for which our labels are missing. For all variables that we model in log-space, we first add 1 to deal with 0-valued observations (except for housing, for which we do not have any 0's).

In [8]:
results_dict = {}
results_dict_test = {}
for label in labels_to_run:

    print("*** Running regressions for: {}".format(label))

    ## Set some label-specific variables
    c = io.get_filepaths(c, label)
    c_app = getattr(c, label)
    sampling_type = c_app["sampling"]  # UAR or POP
    this_save_patt = save_patt.format(
        subset="",
        save_dir=c.fig_dir_prim,
        label=label,
        variable=c_app["variable"],
        sampling=c_app["sampling"],
    )

    # decide wehether to just test the best lambda(s) or all of them
    if fixed_lambda:
        best_lambda_fpath = this_save_patt.format(reg_type="scatter", subset="")
    else:
        best_lambda_fpath = None
    this_lambdas = io.get_lambdas(c, label, best_lambda_fpath=best_lambda_fpath)

    # determine bounds of predictions
    if c_app["logged"]:
        bounds = np.array([c_app["us_bounds_log_pred"]])
    else:
        bounds = np.array([c_app["us_bounds_pred"]])

    ## Get save path
    if (subset_n != slice(None)) or (subset_feat != slice(None)):
        subset_str = "_subset"
    else:
        subset_str = ""
    save_path_validation = this_save_patt.format(reg_type="scatter", subset=subset_str)
    save_path_test = this_save_patt.format(reg_type="testset", subset=subset_str)

    if (not overwrite) and (
        os.path.isfile(save_path_validation) or os.path.isfile(save_path_test)
    ):
        raise OVERWRITE_EXCEPTION

    ## get X, Y, latlon values of training data
    (
        this_X,
        this_X_test,
        this_Y,
        this_Y_test,
        this_latlons,
        this_latlons_test,
    ) = parse.merge_dropna_transform_split_train_test(
        c, label, X[sampling_type], latlons[sampling_type]
    )

    ## subset
    this_X = this_X[subset_n, subset_feat]
    this_X_test = this_X_test[:, subset_feat]
    this_Y = this_Y[subset_n]
    this_latlons = this_latlons[subset_n]

    ## Train model using ridge regression and 5-fold cross-valiation
    ## (number of folds can be adjusted using the argument n_folds)
    print("Training model...")
    import time

    st_train = time.time()
    kfold_results = solve.kfold_solve(
        this_X,
        this_Y,
        solve_function=solver,
        num_folds=c.ml_model["n_folds"],
        return_model=True,
        lambdas=this_lambdas,
        return_preds=True,
        svd_solve=False,
        clip_bounds=bounds,
    )
    print("")

    # get timing
    training_time = time.time() - st_train
    print("Training time:", training_time)

    ## Store the metrics and the predictions from the best performing model
    best_lambda_idx, best_metrics, best_preds = ir.interpret_kfold_results(
        kfold_results, "r2_score", hps=[("lambdas", c_app["lambdas"])]
    )
    best_lambda = this_lambdas[best_lambda_idx]

    ## combine out-of-sample predictions over folds
    preds = np.vstack([solve.y_to_matrix(i) for i in best_preds.squeeze()]).squeeze()
    truth = np.vstack(
        [solve.y_to_matrix(i) for i in kfold_results["y_true_test"].squeeze()]
    ).squeeze()

    # get latlons in same shuffled, cross-validated order
    ll = this_latlons[
        np.hstack([test for train, test in kfold_results["cv"].split(this_latlons)])
    ]

    data = {
        "truth": truth,
        "preds": preds,
        "lon": ll[:, 1],
        "lat": ll[:, 0],
        "best_lambda": best_lambda,
    }

    ## save validation set predictions
    print("Saving validation set results to {}".format(save_path_validation))
    with open(save_path_validation, "wb") as f:
        pickle.dump(data, f)
    results_dict = r2_score(truth, preds)

    ## Get test set predictions
    st_test = time.time()
    holdout_results = solve.single_solve(
        this_X,
        this_X_test,
        this_Y,
        this_Y_test,
        lambdas=best_lambda,
        svd_solve=False,
        return_preds=True,
        return_model=False,
        clip_bounds=bounds,
    )

    # get timing
    test_time = time.time() - st_test
    print("Test set training time:", test_time)

    ## Save test set predictions
    ll = this_latlons_test
    data = {
        "truth": holdout_results["y_true_test"],
        "preds": holdout_results["y_pred_test"][0][0][0],
        "lon": ll[:, 1],
        "lat": ll[:, 0],
    }

    print("Saving test set results to {}".format(save_path_test))
    with open(save_path_test, "wb") as f:
        pickle.dump(data, f)

    ## Store the R2
    results_dict_test[label] = holdout_results["metrics_test"][0][0][0]["r2_score"]
    print("Full reg time", time.time() - st_train)

*** Running regressions for: treecover
Loading labels...
Merging labels and features...
Splitting training/test...
Training model...
on fold (of 5): 1 2 3 4 5 
Training time: 121.46458005905151
Saving validation set results to /data/output/applications/treecover/figures/primary_analysis/outcomes_scatter_obsAndPred_treecover_treecover_CONTUS_16_640_UAR_100000_0_random_features_3_0.data


/root/capsule/code/mosaiks/solve/interpret_results.py:71: UserWarning: Only one value for hyperparameter number 0 supplied.
  "Only one value for hyperparameter number {0} supplied.".format(ix)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Test set training time: 25.147928476333618
Saving test set results to /data/output/applications/treecover/figures/primary_analysis/outcomes_testset_obsAndPred_treecover_treecover_CONTUS_16_640_UAR_100000_0_random_features_3_0.data
Full reg time 146.63663625717163
*** Running regressions for: elevation
Loading labels...
Merging labels and features...
Splitting training/test...
Training model...
on fold (of 5): 1 2 3 4 5 
Training time: 80.77733635902405
Saving validation set results to /data/output/applications/elevation/figures/primary_analysis/outcomes_scatter_obsAndPred_elevation_meters_CONTUS_16_640_UAR_100000_0_random_features_3_0.data


/root/capsule/code/mosaiks/solve/interpret_results.py:71: UserWarning: Only one value for hyperparameter number 0 supplied.
  "Only one value for hyperparameter number {0} supplied.".format(ix)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Test set training time: 24.181716203689575
Saving test set results to /data/output/applications/elevation/figures/primary_analysis/outcomes_testset_obsAndPred_elevation_meters_CONTUS_16_640_UAR_100000_0_random_features_3_0.data
Full reg time 104.98456883430481
*** Running regressions for: population
Loading labels...
Merging labels and features...
Splitting training/test...
Training model...
on fold (of 5): 1 2 3 4 5 
Training time: 58.80880570411682
Saving validation set results to /data/output/applications/population/figures/primary_analysis/outcomes_scatter_obsAndPred_population_log_population_CONTUS_16_640_UAR_100000_0_random_features_3_0.data


...


Training time: 74.96262001991272
Saving validation set results to /data/output/applications/income/figures/primary_analysis/outcomes_scatter_obsAndPred_income_income_CONTUS_16_640_POP_100000_0_random_features_3_0.data
Test set training time: 23.47319269180298
Saving test set results to /data/output/applications/income/figures/primary_analysis/outcomes_testset_obsAndPred_income_income_CONTUS_16_640_POP_100000_0_random_features_3_0.data
Full reg time 98.45929956436157
*** Running regressions for: roads
Loading labels...
Merging labels and features...
Splitting training/test...
Training model...
on fold (of 5): 1 2 3 4 5 
Training time: 81.54102087020874
Saving validation set results to /data/output/applications/roads/figures/primary_analysis/outcomes_scatter_obsAndPred_roads_length_CONTUS_16_640_POP_100000_0_random_features_3_0.data


/root/capsule/code/mosaiks/solve/interpret_results.py:71: UserWarning: Only one value for hyperparameter number 0 supplied.
  "Only one value for hyperparameter number {0} supplied.".format(ix)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Test set training time: 24.554396867752075
Saving test set results to /data/output/applications/roads/figures/primary_analysis/outcomes_testset_obsAndPred_roads_length_CONTUS_16_640_POP_100000_0_random_features_3_0.data
Full reg time 106.11688327789307
*** Running regressions for: housing
Loading labels...
Merging labels and features...
Splitting training/test...
Training model...
on fold (of 5): 1 2 3 4 5 

/root/capsule/code/mosaiks/solve/solve_functions.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  "y_true_test": np.array(kfold_y_test),
/root/capsule/code/mosaiks/solve/solve_functions.py:367: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  "y_true_train": np.array(kfold_y_train),
/root/capsule/code/mosaiks/solve/interpret_results.py:71: UserWarning: Only one value for hyperparameter number 0 supplied.
  "Only one value for hyperparameter number {0} supplied.".format(ix)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarni


Training time: 47.45401883125305
Saving validation set results to /data/output/applications/housing/figures/primary_analysis/outcomes_scatter_obsAndPred_housing_log_price_per_sqft_CONTUS_16_640_POP_100000_0_random_features_3_0.data
Test set training time: 11.320044994354248
Saving test set results to /data/output/applications/housing/figures/primary_analysis/outcomes_testset_obsAndPred_housing_log_price_per_sqft_CONTUS_16_640_POP_100000_0_random_features_3_0.data
Full reg time 58.7882034778595


## Save Table S2

In [9]:
# Get save path
if (subset_n != slice(None)) or (subset_feat != slice(None)):
    subset_str = "_subset"
else:
    subset_str = ""

fn = Path(c.res_dir) / "tables" / "TableS2" / f"TestSetPerformance{subset_str}.csv"

# also save in output folder in case interacting w/ CO capsule w/o access to results
fn_output = Path(c.out_dir) / "cnn_comparison" / f"TestSetR2_mosaiks{subset_str}.csv"

if (not overwrite) and subset_str == "" and fn.isfile():
    raise OVERWRITE_EXCEPTION

fn.parent.mkdir(parents=True, exist_ok=True)
results_df = pd.DataFrame(
    {"Cross-validation R2": results_dict, "Test R2": results_dict_test}
)
results_df.index.name = "label"
results_df.to_csv(fn, index=True)
results_df.to_csv(fn_output, index=True)